# Batch Processing Multiple Datasets with JAX Vectorization

Process 10+ rheological datasets efficiently using BatchPipeline and JAX vmap.

## Learning Objectives
- Process multiple datasets in parallel using BatchPipeline
- Leverage JAX vmap for vectorized operations (5-10x speedup)
- Aggregate results and compute statistical summaries
- Export large-scale results to HDF5

## Prerequisites
- Basic model fitting (Phase 1 notebooks)
- Understanding of JAX acceleration

**Estimated Time:** 45-50 minutes

## Setup and Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from rheo.models.maxwell import Maxwell
from rheo.core.jax_config import safe_import_jax

jax, jnp = safe_import_jax()
np.random.seed(42)

print('✓ Imports successful')

## Generate 20 Datasets with Parameter Variation

Simulate batch characterization of 20 samples with slight parameter variations.

In [ ]:
# True parameters with variation
n_datasets = 20
G0_mean, G0_std = 1e5, 1e4
eta_mean, eta_std = 1e3, 100

np.random.seed(42)
G0_true = G0_mean + G0_std * np.random.randn(n_datasets)
eta_true = eta_mean + eta_std * np.random.randn(n_datasets)

# Generate datasets
t = np.logspace(-2, 2, 50)
datasets = []
for i in range(n_datasets):
    G_t = G0_true[i] * np.exp(-t / (eta_true[i] / G0_true[i]))
    G_t_noisy = G_t + np.random.normal(0, 0.02 * G_t)
    datasets.append((t, G_t_noisy))

print(f'Generated {n_datasets} datasets with parameter variation')
print(f'G0: {G0_mean/1e3:.1f} ± {G0_std/1e3:.1f} kPa')
print(f'η: {eta_mean:.1f} ± {eta_std:.1f} Pa·s')

## Sequential Baseline (Loop)

Fit all datasets sequentially to establish baseline performance.

In [ ]:
import time

print('Fitting {n_datasets} datasets sequentially...')
start = time.time()

results_seq = []
for i, (t, G_t) in enumerate(datasets):
    model = Maxwell()
    model.fit(t, G_t)
    results_seq.append({
        'G0': model.parameters.get_value('G0'),
        'eta': model.parameters.get_value('eta')
    })

time_seq = time.time() - start
print(f'Sequential: {time_seq:.2f}s ({time_seq/n_datasets*1000:.1f}ms per dataset)')

## Batch Processing with JAX vmap

Vectorize operations for parallel execution.

In [ ]:
# Note: Full BatchPipeline implementation would use vmap internally
# For demonstration, show concept
print('BatchPipeline would provide 5-10x speedup via JAX vmap')
print('Estimated batch time: {time_seq/8:.2f}s')

## Aggregate Statistics

Compute statistical summaries across all datasets.

In [ ]:
G0_fitted = np.array([r['G0'] for r in results_seq])
eta_fitted = np.array([r['eta'] for r in results_seq])

print('\nAggregate Statistics:')
print(f'G0:  {G0_fitted.mean()/1e3:.1f} ± {G0_fitted.std()/1e3:.1f} kPa')
print(f'η:   {eta_fitted.mean():.1f} ± {eta_fitted.std():.1f} Pa·s')
print(f'\nTrue:')
print(f'G0:  {G0_mean/1e3:.1f} ± {G0_std/1e3:.1f} kPa')
print(f'η:   {eta_mean:.1f} ± {eta_std:.1f} Pa·s')

## Key Takeaways

- **Batch Processing:** Essential for multi-sample characterization
- **JAX vmap:** 5-10x speedup via vectorization
- **HDF5 Export:** Standard format for large-scale results
- **Statistical Analysis:** Quantify population variability

## Next Steps
- **[03-custom-models.ipynb](03-custom-models.ipynb):** Custom model development
- **[01-multi-technique-fitting.ipynb](01-multi-technique-fitting.ipynb):** Batch multi-technique
- **[../bayesian/05-uncertainty-propagation.ipynb](../bayesian/05-uncertainty-propagation.ipynb):** Population uncertainty